# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Grouping,-Aggregating,-and-Reshaping-Data" data-toc-modified-id="Grouping,-Aggregating,-and-Reshaping-Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Grouping, Aggregating, and Reshaping Data</a></div><div class="lev2 toc-item"><a href="#Discretising,-Grouping,-and-Bar-Charting-Groups" data-toc-modified-id="Discretising,-Grouping,-and-Bar-Charting-Groups-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Discretising, Grouping, and Bar-Charting Groups</a></div><div class="lev2 toc-item"><a href="#One-could-obtain-simple-aggregate-and-statistical-measures-with-methods-we-have-already-seen" data-toc-modified-id="One-could-obtain-simple-aggregate-and-statistical-measures-with-methods-we-have-already-seen-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>One could obtain simple aggregate and statistical measures with methods we have already seen</a></div><div class="lev2 toc-item"><a href="#Grouping-DataFrames" data-toc-modified-id="Grouping-DataFrames-13"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Grouping DataFrames</a></div><div class="lev2 toc-item"><a href="#Exercises" data-toc-modified-id="Exercises-14"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Exercises</a></div>

# Grouping, Aggregating, and Reshaping Data

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import seaborn as sns
sns.set()
sns.set_context("notebook")

In [ ]:
mpl.rcParams['figure.figsize'] = (6.4*1.4, 4.8*1.4)

In [ ]:
pima_df = pd.read_csv('../data/pima-indians-diabetes.csv')
pima_df.sample(3)

In [ ]:
gm_df = pd.read_csv('../data/gapminder.tsv', sep='\t')
gm_df.sample(3)

## Discretising, Grouping, and Bar-Charting Groups 

In [ ]:
pima_df.head()

In [ ]:
# discretising bmi
# Underweight: BMI is less than 18.5.
# Normal weight: BMI is 18.5 to 24.9.
# Overweight: BMI is 25 to 29.9.
# Obese: BMI is 30 or more.
pima_df['bmi_lbl'] = \
    pd.cut(
        pima_df['bmi'],
        bins=[0,18.5,24.9,29.9,pima_df['bmi'].max()],
        labels=['low', 'normal', 'overweight', 'obese'],
        include_lowest=True
    )
pima_df.head(5)

In [ ]:
pima_df['class'].value_counts()

In [ ]:
pima_df.groupby(by=['class', 'bmi_lbl']).size()

In [ ]:
pima_df.groupby(by=['class', 'bmi_lbl']).size().unstack()

In [ ]:
pima_df.groupby(by=['class', 'bmi_lbl']).size().unstack().plot.bar()

## One could obtain simple aggregate and statistical measures with methods we have already seen

In [ ]:
gm_df['year'].count()

In [ ]:
gm_df['lifeExp'].mean()

In [ ]:
gm_df[ ['lifeExp', 'gdpPercap'] ].mean()

In [ ]:
gm_df['continent'].value_counts()

In [ ]:
gm_df['lifeExp'].describe()

In [ ]:
# format strings: https://docs.python.org/3.6/library/string.html#formatstrings
'{:,d}'.format(gm_df.loc[ gm_df['year']==2007, 'pop'].sum())

## Grouping DataFrames

In [ ]:
# one partition is produced for each distinct value of `continent`.
# size() is a method on a groupBy object that returns the size
# (number of rows) of a partition
gm_df.groupby('continent').size()

In [ ]:
# have a look at the split-apply-combine diagram in the Python Data Science Handbook
# https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html#Split,-apply,-combine

In [ ]:
# this is what a grouping object is
groupings = gm_df.groupby('country')
groupings

In [ ]:
groupings.get_group('Argentina')

In [ ]:
# just for teaching/learning purposes, we could iterate through the results
i = iter(groupings)
g = next(i)
g

In [ ]:
g[0]

In [ ]:
g[1]

In [ ]:
# thanks to some hard work by the `pandas` community, one can pass on calls
# to DataFrame methods to each partition by simply chaining them
groupings['lifeExp'].mean().head()

In [ ]:
groupings['lifeExp'].mean().sample(5)

In [ ]:
groupings['lifeExp'].min().sample(5)

In [ ]:
gm_df.loc[ gm_df['country']=='Kenya' ]

In [ ]:
# more than one grouping attribute
gm_df.groupby(['continent', 'year'])[['lifeExp']].mean().head(20)

In [ ]:
mask = (gm_df['year']==2007) & (gm_df['continent'].isin(['Africa', 'Americas' ]))
gm_df.loc[mask, 'lifeExp'].mean()

In [ ]:
# more than one grouping attribute and more than one column
gm_df.groupby(['continent', 'year'])[['lifeExp', 'gdpPercap']].mean().head(20)

In [ ]:
# more than one aggregate?
df = gm_df.groupby('year').agg({'lifeExp': ['min', 'max', 'mean'] })
df

In [ ]:
# a quick look at indexing a column hierarchical index...
df['lifeExp', 'min']

In [ ]:
# different columns and aggregate functions
gm_df.groupby('continent').agg({'lifeExp': 'min', 'gdpPercap': 'max'}).head()

In [ ]:
# combining operations for comparative data science
df_2007_1952_americas = gm_df.loc[
    (gm_df['continent']=='Americas') & (gm_df['year'].isin([2007, 1952]))
]
df_2007_1952_americas.head()

In [ ]:
df_2007_1952_americas.groupby('year')[ ['lifeExp','gdpPercap'] ].mean()

In [ ]:
df = df_2007_1952_americas.groupby('year')[ ['lifeExp','gdpPercap'] ].mean()
df

In [ ]:
# transposition: useful for comparing groups
df.T

In [ ]:
df_2007_vs_1952 = df.T
df_2007_vs_1952

In [ ]:
(df_2007_vs_1952[2007]-df_2007_vs_1952[1952])/df_2007_vs_1952[1952]*100

## Exercises

We'll make use of `seaborn`'s `titanic` dataset; write expressions that compute the information required by the queries below.

In [ ]:
# the titanic dataset is built into `seaborn`.
# The Kaggle website provides some information on it:
# https://www.kaggle.com/c/titanic/data
titanic_df = sns.load_dataset('titanic')
titanic_df.head()

In [ ]:
len(titanic_df.loc[ titanic_df['deck'].isnull(), 'deck'])

In [ ]:
# what is total number of passangers by sex?


In [ ]:
# what is the number of passangers by port of embarkment?


In [ ]:
# what is the mean age by sex?


In [ ]:
# how many passengers travelled alone, by sex?


In [ ]:
# what is the mean fare value by class?


In [ ]:
# how many passengers survived and died, by class?


In [ ]:
# how many passengers survived and died, by sex?


In [ ]:
# what are the highest values for `sibsp` (number of siblings/spouses) and 
# `parch` (number of parents/children)
# by sex?


In [ ]:
# compare the average fear paid by men and women, broken down in
# the three different classes
